## Remove duplicates and profile spectra
This notebooks removes duplicated spectra and removes profile spectra

In [2]:
from matchms.importing.load_from_mgf import load_from_mgf
from tqdm import tqdm

file_name = "results_library_cleaning/cleaned_libraries.mgf"
cleaned_spectra = list(tqdm(load_from_mgf(file_name)))
len(cleaned_spectra)

449721it [01:55, 3892.10it/s]


449721

In [3]:
from matchms.exporting import save_as_mgf
from matchms.importing import load_from_mgf
from matchms.similarity.CosineGreedy import CosineGreedy
from matchms import calculate_scores
from tqdm import tqdm
import numpy as np

unique_inchikeys = {}
# select unique inchikeys
# save the spectrum index
for i,  spectrum in tqdm(enumerate(cleaned_spectra), desc="selecting spectra"):
    inchikey = spectrum.get("inchikey")
    if inchikey is None:
        print("Warning inchikey was None")
    if inchikey in unique_inchikeys:
        unique_inchikeys[inchikey].append(i)
    else:
        unique_inchikeys[inchikey] = [i]
print(len(unique_inchikeys))
non_duplicate_spectra = []
removed_spectra = []
for inchikey in tqdm(unique_inchikeys, desc="looping inchikeys"):
    matching_spectra = []
    for spectrum_idx in unique_inchikeys[inchikey]:
        matching_spectra.append(cleaned_spectra[spectrum_idx])
    scores = calculate_scores(references=matching_spectra,
                              queries=matching_spectra,
                              similarity_function=CosineGreedy())
    scores_array = scores.to_array(name="CosineGreedy_score")
    indices = np.argwhere(scores_array == 1.0)

    # Step 2: Filter out diagonal indices
    non_diagonal_indices = [tuple(index) for index in indices if index[0] != index[1]]

    indexes_removed = []
    for index1, index2 in non_diagonal_indices:
        if index1 not in indexes_removed and index2 not in indexes_removed:
            indexes_removed.append(index1)
    for i, spectrum in enumerate(matching_spectra):
        if i not in indexes_removed:
            non_duplicate_spectra.append(spectrum)
        else:
            removed_spectra.append(spectrum)


selecting spectra: 449721it [00:10, 43685.05it/s]


34938


looping inchikeys: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34938/34938 [53:01<00:00, 10.98it/s]


In [4]:
len(removed_spectra)

22504

## Remove profiled spectra


In [5]:
import logging
import numpy as np
from matchms.Spectrum import Spectrum


logger = logging.getLogger("matchms")


def remove_profiled_spectra(spectrum: Spectrum, mz_window=0.5):
    """Remove profiled spectra

    Spectra are removed if within the mz_window of 0.5 of the highest peak at least 2 peaks next to the main peak are of
    intensity > max_intensity/2.

    Reproduced from MZmine.
    https://github.com/mzmine/mzmine3/blob/master/src/main/java/io/github/mzmine/util/scans/ScanUtils.java#L609
    """
    peaks_n = spectrum.mz.shape[0]
    if peaks_n < 3:
        return spectrum

    # mz_window = spectrum.mz[-1] - spectrum.mz[0] / 1000
    number_of_high_intensity_surounding_peaks = _get_number_of_high_intensity_surounding_peaks(spectrum.intensities,
                                                                                               spectrum.mz, mz_window)
    if number_of_high_intensity_surounding_peaks < 3:
        return spectrum
    logger.info(
        "Spectrum removed because likely profile data."
        "Number of high intensity fragments next to the highest peak = %s.",
        number_of_high_intensity_surounding_peaks)
    return None


def _get_number_of_high_intensity_surounding_peaks(intensities, mz, mz_window):
    base_peak_i = intensities.argmax()

    intensities_within_mz_window = _select_intensities_within_mz_window(intensities, mz, mz_window)
    nr_of_peaks_above_threshold_before_base_peak, nr_of_peaks_above_threshold_after_base_peak = _get_peak_intens_neighbourhood(intensities_within_mz_window)
    base_peak_min_i = base_peak_i - nr_of_peaks_above_threshold_before_base_peak
    base_peak_max_i = base_peak_i + nr_of_peaks_above_threshold_after_base_peak

    number_of_high_intensity_surounding_peaks = base_peak_max_i - base_peak_min_i + 1
    return number_of_high_intensity_surounding_peaks


def _select_intensities_within_mz_window(intensities, mz, mz_span):
    base_peak_i = intensities.argmax()
    within_mz_window = (mz > mz[base_peak_i] - mz_span) & (mz < mz[base_peak_i] + mz_span)
    intensities_within_mz_window = intensities[within_mz_window]
    return intensities_within_mz_window


def _get_peak_intens_neighbourhood(intensities):
    """
    Returns the range of indices around the highest peak that are more than half the intensity of the highest peak.
    """

    base_peak_i = intensities.argmax()
    intensity_threshold = intensities[base_peak_i] / 2

    # Select true for each peak above threshold. Adding False on both ends, to always get a result from np.argmin.
    threshold_mask = np.concatenate([[False], intensities > intensity_threshold, [False]])

    nr_of_peaks_above_threshold_before_base_peak = np.argmin(np.flip(threshold_mask[:base_peak_i + 1]))
    nr_of_peaks_above_threshold_after_base_peak = np.argmin(threshold_mask[base_peak_i + 2:])
    return nr_of_peaks_above_threshold_before_base_peak, nr_of_peaks_above_threshold_after_base_peak

In [6]:
non_profile_spectra = []
for spectrum in tqdm(non_duplicate_spectra):
    spectrum = remove_profiled_spectra(spectrum)
    if spectrum is not None:
        non_profile_spectra.append(spectrum)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 427217/427217 [00:21<00:00, 19949.47it/s]


In [7]:
print(len(non_duplicate_spectra))
print(len(non_profile_spectra))

427217
425604


In [8]:
file_name = "results_library_cleaning/cleaned_libraries_2.mgf"
save_as_mgf(non_profile_spectra, file_name)


## Remove spectra with more than 300 peaks
This can also be done by setting pipeline.processing_queries.parse_and_add_filter((require_number_of_peaks_below_maximum, {"maximum_number_of_fragments": 1000})) to 300 in the previous notebook. 


In [9]:
kept_spectra = []
for spectrum in tqdm(non_profile_spectra):
    if len(spectrum.mz) < 300:
        kept_spectra.append(spectrum)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 425604/425604 [00:06<00:00, 62020.36it/s]


In [12]:
file_name = "results_library_cleaning/cleaned_libraries_3.mgf"
save_as_mgf(kept_spectra, file_name)

## Add massspecgym identifier

In [32]:
spectrum_with_id = []
for i, spectrum_in in enumerate(tqdm(kept_spectra)):
    spectrum = spectrum_in.clone()
    spectrum.set("identifier", f"MassSpecGymID{i+1:0{7}d}")
    spectrum_with_id.append(spectrum)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 414174/414174 [01:46<00:00, 3885.52it/s]


In [34]:
file_name = "results_library_cleaning/cleaned_libraries_4.mgf"
save_as_mgf(spectrum_with_id, file_name)